# Notes:
Does not work well.
I need to try blurring the image so the high level features stay.

#### Run Yolo on input images, save detected objects

In [31]:
import torch
from PIL import Image, UnidentifiedImageError
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # You can change 'yolov5s' to any other YOLOv5 model

def detect_and_crop(image_path, output_path, target_obj):
    # Load image
    try:
        img = cv2.imread(image_path)
        # You may need to convert the color.
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        #img = Image.open(image_path).convert("RGB")  # Ensure the image is in RGB format
        img_np = np.array(img)
    except UnidentifiedImageError:
        print(f"Skipping invalid image file: {image_path}")
        return 0
    except Exception as e:
        print(f"Error processing file {image_path}: {e}")
        return 0
    
    # Perform inference
    results = model(img_np)

    # Extract bounding boxes
    detections = results.xyxy[0].numpy()  # x1, y1, x2, y2, confidence, class
    names = results.names  # Class names
    count = 0
    for i, detection in enumerate(detections):
        x1, y1, x2, y2, conf, cls = map(int, detection[:6])
        
        # Crop the bounding box from the original image
        if names[cls] == target_obj:
            cropped_img = img_np[y1:y2, x1:x2]
            cropped_pil_img = Image.fromarray(cropped_img)
            # os.makedirs(output_path, exist_ok=True)
            cropped_pil_img.save(f"{output_path}_{names[cls]}_crop_{i}.png")
            count += 1

    print(f"Detected and saved {count} cropped images of {target_obj} from {image_path}.")
    return count

# Example usage
img = 'dining-chair-1'
total_detected = 0
# cropping_path = 'wikidata_imgs/raw'
# output_path = 'wikidata_imgs/cropped/'
cropping_path = 'input_imgs/'
output_path = 'cropped_imgs/'
for root, dirs, files in os.walk(cropping_path):
    for file in files:
        file_path = os.path.join(root, file)
        filename = file.split('.')[-2]
        #print(f"Processing file: {file}")
        detected = detect_and_crop(file_path, f"{output_path}{filename}", 'chair')
        total_detected += detected

print(f"Total detected and saved images: {total_detected}")


Using cache found in /home/mark/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-7-25 Python-3.8.10 torch-2.3.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Detected and saved 1 cropped images of chair from input_imgs/office-chair-2.png.
Detected and saved 5 cropped images of chair from input_imgs/dining-chair-2.png.
Detected and saved 2 cropped images of chair from input_imgs/office-chair-1.png.
Detected and saved 1 cropped images of chair from input_imgs/dining-chair-1.png.
Detected and saved 0 cropped images of chair from input_imgs/arm-chair-1.png.
Detected and saved 2 cropped images of chair from input_imgs/arm-chair-2.png.
Total detected and saved images: 11


#### Take detected objects, run similarity matching on wikidata imgs

In [39]:
def match_detections(cropped_imgs_folder, wikidata_imgs_folder, output_folder):
    # Initialize SIFT detector
    sift = cv2.SIFT_create()
    file_name = cropped_imgs_folder.split('/')[-1].split('.')[-2]
    print(f'...Matching {file_name}')
    # Function to read images and convert to grayscale
    def read_image(image_path):
        img = cv2.imread(image_path)
        return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Load all cropped images
    cropped_images = []
    for root, _, files in os.walk(cropped_imgs_folder):
        for file in files:
            cropped_images.append((os.path.join(root, file), read_image(os.path.join(root, file))))

    # Load all wikidata images
    wikidata_images = []
    for root, _, files in os.walk(wikidata_imgs_folder):
        for file in files:
            wikidata_images.append((os.path.join(root, file), read_image(os.path.join(root, file))))

    best_match = None
    highest_score = 0
    
    # Match each cropped image with each wikidata image
    for cropped_path, cropped_img in cropped_images:
        kp1, des1 = sift.detectAndCompute(cropped_img, None)
        for wikidata_path, wikidata_img in wikidata_images:
            kp2, des2 = sift.detectAndCompute(wikidata_img, None)
            
            # Use BFMatcher to find matches
            bf = cv2.BFMatcher()
            matches = bf.knnMatch(des1, des2, k=2)
            
            # Apply ratio test
            good_matches = []
            for m, n in matches:
                if m.distance < 0.75 * n.distance:
                    good_matches.append(m)
            
            if len(good_matches) > highest_score:
                highest_score = len(good_matches)
                best_match = (cropped_path, wikidata_path, kp1, kp2, good_matches, cropped_img, wikidata_img)

    if best_match:
        cropped_path, wikidata_path, kp1, kp2, good_matches, cropped_img, wikidata_img = best_match
        print(f"Best match: {cropped_path} and {wikidata_path} with {highest_score} matches.")
        
        # Draw matches
        matched_img = cv2.drawMatches(cropped_img, kp1, wikidata_img, kp2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        
        # Save the image with matched features
        os.makedirs(output_folder, exist_ok=True)
        matched_img_path = os.path.join(output_folder, f'{file_name}.png')
        cv2.imwrite(matched_img_path, matched_img)
        print(f"Matched image saved at {matched_img_path}")

matching_path = 'cropped_imgs/'

for root, dirs, files in os.walk(matching_path):
    for file in files:
        file_path = os.path.join(root, file)
        filename = file.split('.')[-2]
        print(f"Processing file: {file_path}")
        match_detections(f'{file_path}', 'wikidata_imgs/cropped/', 'matched_imgs/')

Processing file: cropped_imgs/dining-chair-2_chair_crop_11.png
...Matching dining-chair-2_chair_crop_11
Processing file: cropped_imgs/office-chair-2_chair_crop_0.png
...Matching office-chair-2_chair_crop_0
Processing file: cropped_imgs/arm-chair-2_chair_crop_6.png
...Matching arm-chair-2_chair_crop_6
Processing file: cropped_imgs/dining-chair-2_chair_crop_0.png
...Matching dining-chair-2_chair_crop_0
Processing file: cropped_imgs/dining-chair-2_chair_crop_3.png
...Matching dining-chair-2_chair_crop_3
Processing file: cropped_imgs/arm-chair-2_chair_crop_0.png
...Matching arm-chair-2_chair_crop_0
Processing file: cropped_imgs/office-chair-1_chair_crop_0.png
...Matching office-chair-1_chair_crop_0
Processing file: cropped_imgs/dining-chair-1_chair_crop_1.png
...Matching dining-chair-1_chair_crop_1
Processing file: cropped_imgs/office-chair-1_chair_crop_1.png
...Matching office-chair-1_chair_crop_1
Processing file: cropped_imgs/dining-chair-2_chair_crop_1.png
...Matching dining-chair-2_cha

In [40]:
import os
import cv2
import numpy as np
from PIL import Image, UnidentifiedImageError

# Function to read an image and convert to grayscale
def read_image(image_path):
    img = cv2.imread(image_path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Function to find the best match for a given image
def find_best_match(file_path, wikidata_imgs_folder, output_folder):
    # Initialize SIFT detector
    sift = cv2.SIFT_create()

    # Read the input image
    try:
        input_img = read_image(file_path)
        kp1, des1 = sift.detectAndCompute(input_img, None)
    except Exception as e:
        print(f"Error reading input image {file_path}: {e}")
        return

    # Load all wikidata images
    wikidata_images = []
    for root, _, files in os.walk(wikidata_imgs_folder):
        for file in files:
            wikidata_images.append((os.path.join(root, file), read_image(os.path.join(root, file))))

    best_match = None
    highest_score = 0

    # Match the input image with each wikidata image
    for wikidata_path, wikidata_img in wikidata_images:
        kp2, des2 = sift.detectAndCompute(wikidata_img, None)

        # Use BFMatcher to find matches
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(des1, des2, k=2)

        # Apply ratio test
        good_matches = []
        for m, n in matches:
            if m.distance < 0.75 * n.distance:
                good_matches.append(m)

        if len(good_matches) > highest_score:
            highest_score = len(good_matches)
            best_match = (wikidata_path, kp1, kp2, good_matches, input_img, wikidata_img)

    if best_match:
        wikidata_path, kp1, kp2, good_matches, input_img, wikidata_img = best_match
        print(f"Best match: {file_path} and {wikidata_path} with {highest_score} matches.")

        # Draw matches
        matched_img = cv2.drawMatches(input_img, kp1, wikidata_img, kp2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

        # Save the image with matched features
        os.makedirs(output_folder, exist_ok=True)
        matched_img_path = os.path.join(output_folder, os.path.basename(file_path).split('.')[0] + '_match.png')
        cv2.imwrite(matched_img_path, matched_img)
        print(f"Matched image saved at {matched_img_path}")


for root, dirs, files in os.walk(matching_path):
    for file in files:
        file_path = os.path.join(root, file)
        filename = file.split('.')[-2]
        print(f"Processing file: {file_path}")
        find_best_match(file_path, 'wikidata_imgs/cropped/', 'matched_imgs/')


Processing file: cropped_imgs/dining-chair-2_chair_crop_11.png
Best match: cropped_imgs/dining-chair-2_chair_crop_11.png and wikidata_imgs/cropped/commons-image-office-chair-1chair_crop_1.png with 11 matches.
Matched image saved at matched_imgs/dining-chair-2_chair_crop_11_match.png
Processing file: cropped_imgs/office-chair-2_chair_crop_0.png
Best match: cropped_imgs/office-chair-2_chair_crop_0.png and wikidata_imgs/cropped/commons-image-office-chair-1chair_crop_1.png with 26 matches.
Matched image saved at matched_imgs/office-chair-2_chair_crop_0_match.png
Processing file: cropped_imgs/arm-chair-2_chair_crop_6.png
Processing file: cropped_imgs/dining-chair-2_chair_crop_0.png
Best match: cropped_imgs/dining-chair-2_chair_crop_0.png and wikidata_imgs/cropped/commons-image-office-chair-1chair_crop_3.png with 1 matches.
Matched image saved at matched_imgs/dining-chair-2_chair_crop_0_match.png
Processing file: cropped_imgs/dining-chair-2_chair_crop_3.png
Best match: cropped_imgs/dining-ch

In [42]:
import os
import cv2
import numpy as np
from PIL import Image, UnidentifiedImageError

# Function to read an image and convert to grayscale
def read_image(image_path):
    img = cv2.imread(image_path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Function to find the best match for a given image
def find_best_match(file_path, wikidata_imgs_folder, output_folder):
    # Initialize ORB detector
    orb = cv2.ORB_create()

    # Read the input image
    try:
        input_img = read_image(file_path)
        kp1, des1 = orb.detectAndCompute(input_img, None)
    except Exception as e:
        print(f"Error reading input image {file_path}: {e}")
        return

    # Load all wikidata images
    wikidata_images = []
    for root, _, files in os.walk(wikidata_imgs_folder):
        for file in files:
            wikidata_images.append((os.path.join(root, file), read_image(os.path.join(root, file))))

    best_match = None
    highest_score = 0

    # FLANN parameters
    FLANN_INDEX_LSH = 6
    index_params = dict(algorithm=FLANN_INDEX_LSH, table_number=6, key_size=12, multi_probe_level=1)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Match the input image with each wikidata image
    for wikidata_path, wikidata_img in wikidata_images:
        kp2, des2 = orb.detectAndCompute(wikidata_img, None)

        # Use FLANN matcher to find matches
        matches = flann.knnMatch(des1, des2, k=2)

        # Apply ratio test
        good_matches = []
        for match in matches:
            if len(match) == 2:
                m, n = match
                if m.distance < 0.75 * n.distance:
                    good_matches.append(m)

        if len(good_matches) > highest_score:
            highest_score = len(good_matches)
            best_match = (wikidata_path, kp1, kp2, good_matches, input_img, wikidata_img)

    if best_match:
        wikidata_path, kp1, kp2, good_matches, input_img, wikidata_img = best_match
        print(f"Best match: {file_path} and {wikidata_path} with {highest_score} matches.")

        # Draw matches
        matched_img = cv2.drawMatches(input_img, kp1, wikidata_img, kp2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

        # Save the image with matched features
        os.makedirs(output_folder, exist_ok=True)
        matched_img_path = os.path.join(output_folder, os.path.basename(file_path).split('.')[0] + '_match.png')
        cv2.imwrite(matched_img_path, matched_img)
        print(f"Matched image saved at {matched_img_path}")


for root, dirs, files in os.walk(matching_path):
    for file in files:
        file_path = os.path.join(root, file)
        filename = file.split('.')[-2]
        print(f"Processing file: {file_path}")
        find_best_match(file_path, 'wikidata_imgs/cropped/', 'matched_imgs/')


Processing file: cropped_imgs/dining-chair-2_chair_crop_11.png
Best match: cropped_imgs/dining-chair-2_chair_crop_11.png and wikidata_imgs/cropped/commons-image-chair-2chair_crop_0.png with 9 matches.
Matched image saved at matched_imgs/dining-chair-2_chair_crop_11_match.png
Processing file: cropped_imgs/office-chair-2_chair_crop_0.png
Best match: cropped_imgs/office-chair-2_chair_crop_0.png and wikidata_imgs/cropped/commons-image-chair-2chair_crop_1.png with 12 matches.
Matched image saved at matched_imgs/office-chair-2_chair_crop_0_match.png
Processing file: cropped_imgs/arm-chair-2_chair_crop_6.png
Best match: cropped_imgs/arm-chair-2_chair_crop_6.png and wikidata_imgs/cropped/commons-image-office-chair-1chair_crop_0.png with 2 matches.
Matched image saved at matched_imgs/arm-chair-2_chair_crop_6_match.png
Processing file: cropped_imgs/dining-chair-2_chair_crop_0.png
Processing file: cropped_imgs/dining-chair-2_chair_crop_3.png
Processing file: cropped_imgs/arm-chair-2_chair_crop_0.